In [2]:
import re

In [27]:
import re

the_script = 'disjdisjdioas<script>this is the reason why all the people did not care about.</script>dsajdiosajdiosajdoisa<script>time to go</script>'

# 使用正则表达式匹配完整的<script>标签及其内容
match = re.search(r'<script(.*?)script>', the_script, re.IGNORECASE | re.DOTALL)

if match:
    # 输出匹配的<script>标签及其内容
    print(match.group())
else:
    print("没有找到匹配的<script>标签")

<script>this is the reason why all the people did not care about.</script>


In [1]:
def get_the_data(job_name,city_name):
    import pandas
    from requests_html import HTMLSession
    import json
    import pandas as pd
    import requests
    用户输入职位 = job_name
    用户输入城市 = city_name
    城市编码 = {
    '北京':'010',
    '上海':'020',
    '广州':'050020',
    '深圳':'050090',

}
    url = "https://apic.liepin.com/api/com.liepin.searchfront4c.pc-search-job"
    payload = {
    "data": {
        "mainSearchPcConditionForm": {
            "city": 城市编码[用户输入城市],
            "dq": 城市编码[用户输入城市],
            "pubTime": "",
            "currentPage": 0,
            "pageSize": 40,
            "key": str(用户输入职位),
            "suggestTag": "",
            "workYearCode": "0",
            "compId": "",
            "compName": "",
            "compTag": "",
            "industry": "",
            "salary": "",
            "jobKind": "",
            "compScale": "",
            "compKind": "",
            "compStage": "",
            "eduLevel": ""
        },
        "passThroughForm": {
            "scene": "input",
            "skId": "",
            "fkId": "",
            "ckId": "h2c8pxojavrmo1w785z7ueih2ybfpux8",
            "suggest": None
        }
    }
}
    session = HTMLSession()
    headers = {your headers}
    r = session.post(url, data=json.dumps(payload), headers=headers)
    response_data = r.json()
    page = response_data['data']['pagination']['totalPage']
    response_df = []
    for i in range(page): # 需要判断页面的数据有多少页
        payload['data']['mainSearchPcConditionForm']['currentPage']=i
        # send a POST request with headers
        r = requests.post(url, data=json.dumps(payload), headers=headers)
    response_data = r.json()
    print(response_data)
    df = pd.json_normalize(response_data['data']['data']['jobCardList'])
    response_df.append(df)
    df = pd.concat(response_df)
    key = payload['data']['mainSearchPcConditionForm']['key']
    df.to_excel('liepin2.xlsx')
    df = pd.read_excel('liepin2.xlsx')
    df

    df_PM_gz =  df[['job.labels','job.refreshTime','job.title','job.salary','job.dq','job.topJob','job.requireWorkYears','job.requireEduLevel','comp.compStage','comp.compName','comp.compIndustry','comp.compScale']]
    df_PM_gz

    df_PM_gz['job.dq'].value_counts()
    地区 = [ df_PM_gz['job.dq'].value_counts().index.tolist()[i].split('-')[1]\
             for i,v in enumerate(df_PM_gz['job.dq'].value_counts().index.tolist()) if '-' in v]
    地区
    岗位个数 = [ df_PM_gz['job.dq'].value_counts().values.tolist()[i] for i,v in enumerate(df_PM_gz['job.dq'].value_counts().index.tolist()) if "-" in v]
    岗位个数
    df_PM_gz['job.title'][df_PM_gz['job.title'].str.contains('（')].str.split('（').apply(lambda x: x[0])
    df_job_title = df_PM_gz['job.title'].apply(lambda x: x.split('（')[0].split('/')[0].split('(')[0]).value_counts()
    df_job_title.index.tolist()
    len(df_job_title.index.tolist())
    df_job_title.values.tolist()
    df_PM_gz['job.title'].value_counts()
    PM_title_words = [(df_job_title.index.tolist()[i], df_job_title.values.tolist()[i]) for i in
                      range(1, len(df_job_title.index.tolist()))]
    PM_title_words
    df_PM_gz['job.labels']
    df_PM_gz['job.labels'].values
    df_PM_gz['job.labels'].apply(lambda x: eval(x)).tolist()
    PM_labels_list = [j for i in df_PM_gz['job.labels'].apply(lambda x: eval(x)).tolist() for j in i]
    PM_labels_list
    PM_labels_words = [(i, PM_labels_list.count(i)) for i in set(PM_labels_list)]
    PM_labels_words
    df_PM_gz = df_PM_gz.rename(columns={
        'job.labels': '职位标签',
        'job.refreshTime': '职位更新时间',
        'job.title': '职位',
        'job.salary': '薪资',
        'job.dq': '地区',
        'job.topJob': '是否top职位',
        'job.requireWorkYears': '工作年限',
        'job.requireEduLevel': '学历',
        'comp.compStage': '公司融资情况',
        'comp.compName': '公司名称',
        'comp.compIndustry': '行业',
        'comp.compScale': '规模'
    })
    df_PM_gz
    非薪资面议 = df_PM_gz[~df_PM_gz['薪资'].str.contains("面议|元/天")]
    非薪资面议
    非薪资面议_detail = 非薪资面议['薪资'].apply(lambda x: x.split('薪')[0].split('·')).tolist()
    非薪资面议_detail
    平均薪资 = [(int(i[0].split('-')[0]) + int(i[0].split('-')[1].split('k')[0])) / 2 \
                    if len(i) == 1 else round(
        (int(i[0].split('-')[0]) + int(i[0].split('-')[1].split('k')[0])) / 2 * int(i[1]) / 12, 1) \
                for i in 非薪资面议_detail]
    平均薪资
    非薪资面议['平均薪资'] = 平均薪资
    分地区_平均薪资 = 非薪资面议.groupby('地区').agg({'平均薪资': 'median'})
    分地区_平均薪资_values = [round(i[0], 1) for i in 分地区_平均薪资.values.tolist()]
    分地区_平均薪资_values
    分地区_平均薪资_index = 分地区_平均薪资.index.tolist()
    分地区_平均薪资_index

    return 地区,岗位个数,PM_title_words,分地区_平均薪资_index,分地区_平均薪资_values

In [2]:
get_the_data('产品经理','深圳')

{'flag': 1, 'data': {'data': {'jobCardList': [{'job': {'labels': ['IT互联网', '计算机软件'], 'link': 'https://www.liepin.com/a/50372563.shtml', 'dq': '深圳', 'title': '高级产品经理', 'requireWorkYears': '5-10年', 'advViewFlag': False, 'topJob': False, 'jobId': '50372563', 'jobKind': '1', 'refreshTime': '20231123094057', 'salary': '35-40k·13薪', 'requireEduLevel': '学历不限', 'dataPromId': 'd_sfrom=search_prime&d_ckId=null&d_curPage=9&d_pageSize=40&d_headId=null&d_posi=0&skId=h2c8pxojavrmo1w785z7ueih2ybfpux8&fkId=h2c8pxojavrmo1w785z7ueih2ybfpux8&ckId=h2c8pxojavrmo1w785z7ueih2ybfpux8&sfrom=search_job_pc&curPage=9&pageSize=40&index=0'}, 'recruiter': {'recruiterPhoto': '5f8fa3bddfb13a7dee343d7608u.png', 'imId': '61783666c13f840ac6974481676d4b89', 'imUserType': '2', 'recruiterName': '赵女士', 'recruiterTitle': '猎头顾问', 'chatted': False, 'recruiterId': 'd631106e2002ed09d5427bf727a6f471'}, 'comp': {'link': '', 'compScale': '10000人以上', 'compStage': '不需要融资', 'compLogo': '60e2fe0bf3df194a3c48adb502u.png', 'compName': '某深

C:\Users\17728\AppData\Local\Temp\ipykernel_19724\705357139.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  非薪资面议['平均薪资'] = 平均薪资


(['福田区', '南山区', '龙岗区', '龙华区', '光明区', '宝安区'],
 [12, 9, 3, 3, 2, 1],
 [('产品经理', 3),
  ('软件产品经理', 2),
  ('助理产品经理', 1),
  ('OA产品经理', 1),
  ('OA高级产品经理', 1),
  ('数据产品经理', 1),
  ('数据中台', 1),
  ('财富管理数字化产品经理岗', 1),
  ('CAD', 1),
  ('产品经理 ', 1),
  ('交易服务系统产品经理', 1),
  ('化妆品产品经理', 1),
  ('视频会议高级产品经理', 1),
  ('高级互联网产品经理', 1),
  ('B端产品经理', 1),
  ('', 1),
  ('平台产品经理', 1),
  ('高级软件产品经理', 1),
  ('高级IT产品经理-HR', 1),
  ('数字化平台产品经理', 1),
  ('金融产品经理', 1),
  ('券商系统高级产品经理', 1),
  ('物流产品经理', 1),
  ('产品经理-SAAS渠道类', 1),
  ('HR系统产品经理', 1),
  ('小家电产品经理', 1),
  ('资深产品经理', 1),
  ('整机产品经理', 1),
  ('无忧行app-商城产品经理', 1),
  ('动力电池Pack 产品高级经理', 1),
  ('动力电池Pack产品经理', 1),
  ('PM顾问', 1)],
 ['深圳', '深圳-光明区', '深圳-南山区', '深圳-宝安区', '深圳-福田区', '深圳-龙华区', '深圳-龙岗区'],
 [49.0, 40.8, 37.5, 46.7, 30.0, 35.2, 29.8])

In [3]:
import time
time.localtime()
output_time = str(time.localtime().tm_year) + '-' \
              + str(time.localtime().tm_mon) + '-' \
              + str(time.localtime().tm_mday)
output_time

'2023-12-13'